In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA

I will build several classification models to distinguish between two related classes of cancer, acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML), using gene expression measurements. Each row in this file corresponds to a tumor tissue sample from a patient with one of the two forms of leukemia. 

The first column contains Cancer_type: 0 = ALL class and 1 = AML class
Columns 2-7130 contain expression levels of 7129 genes recorded from each tissue sample
The last column Cancer_subtype additionally distinguishes between two subtypes of ALL, subtype T and subtype B (used in problem 5): 0 = ALL subtype T, 1 = ALL subtype B, 2 = type AML.

## Data Preparation

In [ ]:
np.random.seed(109)
#zf = zipfile.ZipFile('data/hw5_genes_multiclass.csv.zip') 
df = pd.read_csv('/Users/phili/hw5_genes_multiclass.csv')
X = df.drop(['Cancer_type','Cancer_subtype'], axis=1)
X_train, X_test, y_train, y_test, y2_train, y2_test  = train_test_split(
    X, df.Cancer_type, df.Cancer_subtype, test_size=0.25, random_state = 109,
    stratify = df.Cancer_subtype)

print(df.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(df.Cancer_type.value_counts(normalize=True))

In [ ]:
x_train = (X_train - X_train.min())/(X_train.max() - X_train.min()) 
x_test = (X_test - X_train.min())/(X_train.max() - X_train.min()) 
x_train.head()

In [ ]:
correlations = []
for column in x_train.columns:
    correlations.append(np.corrcoef(x_train[column], y_train)[1][0])

In [ ]:
top_names = []
abs_correlations = list(map(abs, correlations))
sorted_list = sorted(abs_correlations)
top_10 = sorted_list[-10:]
for num in top_10:
    index = abs_correlations.index(num)
    top_names.append(x_train.columns[index])
print(f'The top 10 predictors based on simple correlations are: {top_names[0:10]}')

In [ ]:
best_num = max(top_10)
best_num_index = top_10.index(best_num)
best_predictor = top_names[best_num_index]
print(f'The best predictor based on simple correlations is {best_predictor}')

new_df = x_train
new_df['cancer'] = y_train.values
new_df_test = x_test
new_df_test['cancer'] = y_test.values

plt.figure()
plt.hist(new_df[new_df['cancer'] == 1].X95735_at, bins = 100, stacked=True, density = True, label='AML')
plt.hist(new_df[new_df['cancer'] == 0].X95735_at, bins = 100, stacked=True, density = True, label='ALL')
plt.title('Distribution of X95735_at in Training set')
plt.legend()
plt.show()

plt.figure()
plt.hist(new_df_test[new_df_test['cancer'] == 1].X95735_at, bins=50, stacked=True, density = True, label='AML')
plt.hist(new_df_test[new_df_test['cancer'] == 0].X95735_at, bins=50, stacked=True, density = True, label='ALL')
plt.title('Distribution of X95735_at in Test set')
plt.legend()
plt.show()

## Baseline Model

In [ ]:
cancer_list = []
cancer_list_test = []
for rows in range(0, 564):
    if new_df['X95735_at'].values[rows] > 0.4:
        cancer_list.append(0)
    else:
        cancer_list.append(1)

for rows in range(len(new_df_test)):
    if new_df_test['X95735_at'].values[rows] > 0.4:
        cancer_list_test.append(0)
    else:
        cancer_list_test.append(1)
        
train_score = round(accuracy_score(y_train, cancer_list),3)
test_score = round(accuracy_score(y_test, cancer_list_test),3)
print('0.4 will be used as a classification threshold.')
print(f'The train score is {train_score}')
print(f'The test score is {test_score}')

## Simple Model - Single Predictor

In [ ]:
x = x_train[[best_predictor]]
y = y_train
x_t = x_test[[best_predictor]]
y_t = y_test

logit1 = LogisticRegression(penalty='none', max_iter = 1000)

logit1.fit(x,y)

train_accuracy = logit1.score(x, y)
test_accuracy = logit1.score(x_t, y_t)
print(f'The train accuracy for Logit 1 is {round(train_accuracy,3)}')
print(f'The test accuracy for Logit 1 is {round(test_accuracy, 3)}')

## Model with all predictors

In [ ]:
predictors = x_train.columns
x = x_train
y = y_train
x_t = x_test
y_t = y_test

logit2 = LogisticRegression(penalty='none', max_iter = 1000)

logit2.fit(x,y)

train_accuracy = logit2.score(x, y)
test_accuracy = logit2.score(x_t, y_t)
print(f'The train accuracy for Logit 2 is {round(train_accuracy,3)}')
print(f'The test accuracy for Logit 2 is {round(test_accuracy, 3)}')

The coefficients between the two models disagree. This suggests that there may some covariance between the genes in the dataset. It may be the case that other genes covary with the best predictor and therefore some of the predictive power of the best predictor is removed in the multiple logistic regression because there are so many other predictors. 

Indeed, because the multiple logistic regression involves significant overfitting, it becomes difficult to interpret the coefficients.

## Regularised Model

In [ ]:
Cs = [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3,1e4]
cv = 5
penalty = 'l1'
solver = 'liblinear'

logit_lasso = LogisticRegressionCV(
    Cs=Cs, cv=cv, penalty=penalty, solver='liblinear'
).fit(x, y)

logit_lasso_score_train = round(logit_lasso.score(x, y),3)
logit_lasso_score_test = round(logit_lasso.score(x_t, y_t), 3)

print('The train accuracy is', logit_lasso_score_train)
print('The test accuracy is', logit_lasso_score_test)

In [ ]:
lasso_coefs = list(logit_lasso.coef_[0])
lasso_coefs_abs = list(map(abs, lasso_coefs))
sorted_coefs = sorted(lasso_coefs_abs)
important_coefs = sorted_coefs[-200:]
most_important_coefs = [i for i in important_coefs if i > 0]
num_important_predictors = len(most_important_coefs)
print(f'There are {num_important_predictors} predictors with a coefficient over 0')

## Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scale_transformer = StandardScaler(copy=True).fit(x_train)
x_train_scaled = scale_transformer.transform(x_train)
x_test_scaled = scale_transformer.transform(x_test)

pca = PCA().fit(x_train_scaled)

# transforming the dataframe
x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

print('Dimensions of transformed x_train:', x_train_pca.shape)
print('Dimensions of transformed x_test', x_test_pca.shape)

In [ ]:
colors = ['r','c']
label_text = ["ALL", "AHL"]

# and we loop over the different groups
for cur_quality in [0,1]:
    cur_df = x_train_pca[y_train==cur_quality]
    plt.scatter(cur_df[:,0], cur_df[:,1], c = colors[cur_quality], label=label_text[cur_quality])

# all plots need labels
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimention 2")
plt.legend();

Visualising using PCA allows us to visualise high-dimensional space in low-dimensional space. It would be very difficult to visualise 7129 dimensions, however using PCA we can visualise high-dimensional datasets. This gives a much faster understanding of the nature of the way in which predictors can distinguish between the two classes.

In [ ]:
pca_2_variance_expl = pca.explained_variance_ratio_[0:1]
v_expl = round(pca_2_variance_expl[0],3)
fig, ax = plt.subplots(ncols=2, figsize=(20,6))
ax1,ax2 = ax.ravel()

ratio = pca.explained_variance_ratio_
ax1.bar(range(len(ratio)), ratio, color='blue', alpha=0.8)
ax1.plot(range(1,len(ratio)+1), ratio, 'o-')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Explained Variance Ratio PCA', fontsize=20)
ax1.set_xlabel('PCA');

ratio = pca.explained_variance_ratio_
ax2.plot(range(1,len(ratio)+1), np.cumsum(ratio), 'o-')
ax2.set_title('Cumulative Sum of Explained Variance Ratio PCA', fontsize=20)
ax2.set_ylabel('Cumulative Sum of Explained Variance Ratio');
ax2.set_xlabel('PCA');

print('Roughly 250 PCs are needed to explain 90% of the variability in the predictors')
print(f'The amount of variance in predictors explained by the first two principal components is {v_expl}')

## Logistic Regression + PCA

I cycle through models using increasing numbers of principal components.

In [ ]:
#Using 2 PCs
pca_2 = PCA(n_components=2).fit(x_train_scaled)
x_train_pca_2 = pca_2.transform(x_train_scaled)
x_test_pca_2 = pca_2.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegression(C=100).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 50 PCs
pca_50 = PCA(n_components=50).fit(x_train_scaled)
x_train_pca_2 = pca_50.transform(x_train_scaled)
x_test_pca_2 = pca_50.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegression(C=100, max_iter=500).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 250 PCs
pca_250 = PCA(n_components=250).fit(x_train_scaled)
x_train_pca_2 = pca_250.transform(x_train_scaled)
x_test_pca_2 = pca_250.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegression(C=100, max_iter=10000).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

We see that the accuracy on the testing set improves with an increase in PCAs from 2 to 50, however, decreases with use of 250 PCAs. The training set accuracy also increases with an increased number of PCs from 2 to 50. The training set, however, reaches 100% accuracy with 250 PCs, which suggests overfitting.

## Use cross-validation to calculate the best number of principal components

In [ ]:
#Using 2 PCs and CV
pca_2_cv = PCA(n_components=2).fit(x_train_scaled)
x_train_pca_2 = pca_2_cv.transform(x_train_scaled)
x_test_pca_2 = pca_2_cv.transform(x_test_scaled)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=1000, cv=10).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 50 PCs and CV
pca_50 = PCA(n_components=50).fit(x_train_scaled)
x_train_pca_2 = pca_50.transform(x_train_scaled)
x_test_pca_2 = pca_50.transform(x_test_scaled)


#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=1000, cv=10).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 250 PCs and cv
pca_250 = PCA(n_components=250).fit(x_train_scaled)
x_train_pca_2 = pca_250.transform(x_train_scaled)
x_test_pca_2 = pca_250.transform(x_test_scaled)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=10000, cv=10).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 500 PCs and cv
pca_500 = PCA(n_components=500).fit(x_train_scaled)
x_train_pca_2 = pca_500.transform(x_train_scaled)
x_test_pca_2 = pca_500.transform(x_test_scaled)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=10000, cv=10).fit(x_train_pca_2, y_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y_train, y_pred_train)*100
test_score = accuracy_score(y_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

This suggests that I may have got the number of PCs that explain 90% accuracy wrong. The accuracy is not increasing if more than 250 PCs are used.

## Use multinomial logistic regression models to predict cancer subtype

In [ ]:
#Ridge-like Multinomial with 2 PCs
pca = PCA(n_components=2).fit(x_train_scaled)
x_train_1 = pca.transform(x_train_scaled)
x_test_1 = pca.transform(x_test_scaled)

logit_ridge = LogisticRegression(multi_class='multinomial', max_iter=1000)
logit_ridge.fit(x_train_1,y2_train) 
                                 
y_pred_train = logit_ridge.predict(x_train_1)
y_pred_test = logit_ridge.predict(x_test_1)

train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Ridge-line Multinomial + Quadratic and interaction terms
pca = PCA(n_components=2).fit(x_train_scaled)
x_train = pca.transform(x_train_scaled)
x_test = pca.transform(x_test_scaled)

x_train = PolynomialFeatures(degree=2,include_bias=False).fit_transform(x_train)
x_test = PolynomialFeatures(degree=2,include_bias=False).fit_transform(x_test)

#pca_train_df = pd.DataFrame(x_train, columns=[['PCA1' , 'PCA2']])

#pca_train_df['interaction'] = np.sum(pca_train_df['PCA1'] * pca_train_df['PCA2'])
#pca_train_df['quad_1'] = np.sum(pca_train_df[['PCA1']] ** 2)
#pca_train_df['quad_2'] = np.sum(pca_train_df[['PCA2']] ** 2)

#pca_test_df[['interaction']] = np.sum(pca_test_df[['PCA1']] * pca_test_df[['PCA2']])
#pca_test_df[['quad_1']] = np.sum(pca_test_df[['PCA1']] ** 2)
#pca_test_df[['quad_2']] = np.sum(pca_test_df[['PCA2']] ** 2)

logit_ridge_i = LogisticRegression(multi_class='multinomial', max_iter=1000)
logit_ridge_i.fit(x_train,y2_train) 
#logit_ridge_i.fit(pca_train_df[['PCA1','PCA2', 'interaction', 'quad_1', 'quad_2']],y2_train) 
                                 
y_pred_train = logit_ridge_i.predict(x_train)
y_pred_test = logit_ridge_i.predict(x_test)

train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

## Plot decision boundaries

In [ ]:
pca_test_df = pd.DataFrame(x_test, columns=[['PCA1' , 'PCA2', 'interaction', 'quad_1', 'quad_2']])
colors = ['r','c','b']
label_text = ["ALL Type T", "ALL Type B", "AHL"]

vec1 = x_test_1[:,0]
vec2 = x_test_1[:,1]

x1_range = vec1.max() - vec2.min()
x2_range = vec1.max() - vec2.min()
x1_min, x1_max = vec1.min()-0.1*x1_range, vec1.max() +0.1*x1_range
x2_min, x2_max = vec2.min()-0.1*x2_range, vec2.max() + 0.1*x2_range

step = .05 
x1x, x2x = np.meshgrid(np.arange(x1_min, x1_max, step), np.arange(x2_min, x2_max, step))
y_hat_multi = logit_ridge.predict(np.c_[x1x.ravel(), x2x.ravel()])

plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimention 2")
plt.pcolormesh(x1x, x2x, y_hat_multi.reshape(x1x.shape), cmap=plt.cm.Paired,alpha = 0.5)
for cur_quality in [0,1,2]:
    cur_df = x_test_1[y2_test==cur_quality]
    plt.scatter(cur_df[:,0], cur_df[:,1], c = colors[cur_quality], label=label_text[cur_quality], cmap=plt.cm.Paired)
plt.legend();


In [ ]:
vec3 = x_test[:,0]
vec4 = x_test[:,1]
vec5 = x_test[:,2]
vec6 = x_test[:,3]
vec7 = x_test[:,4]

x3_range = vec3.max() - vec3.min()
x4_range = vec4.max() - vec4.min()
x5_range = vec5.max() - vec5.min()
x6_range = vec6.max() - vec6.min()
x7_range = vec7.max() - vec7.min()


x3_min, x3_max = vec3.min()-0.1*x3_range, vec3.max() +0.1*x3_range
x4_min, x4_max = vec4.min()-0.1*x4_range, vec4.max() + 0.1*x4_range
x5_min, x5_max = vec5.min()-0.1*x5_range, vec5.max() + 0.1*x5_range
x6_min, x6_max = vec6.min()-0.1*x6_range, vec6.max() + 0.1*x6_range
x7_min, x7_max = vec7.min()-0.1*x7_range, vec7.max() + 0.1*x7_range

x3x, x4x = np.meshgrid(np.arange(x3_min, x3_max, step), np.arange(x4_min, x4_max, step))
x5x = x3x**2
x6x = x4x**2
x7x = x3x * x4x
y_hat_poly = logit_ridge_i.predict(np.c_[x3x.ravel(), x4x.ravel(), x5x.ravel(), x6x.ravel(), x7x.ravel()])

#x1x
#x2x
#x1x**2


plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimention 2")
plt.pcolormesh(x3x, x4x, y_hat_poly.reshape(x3x.shape), cmap=plt.cm.Paired,alpha = 0.5)
# and we loop over the different groups
for cur_quality in [0,1,2]:
    cur_df = x_test[y2_test==cur_quality]
    plt.scatter(cur_df[:,0], cur_df[:,1], c = colors[cur_quality], label=label_text[cur_quality], cmap=plt.cm.Paired)
plt.legend();

## Using cross-validation to determine best number of principal components for multi-class prediction

In [ ]:
#Using 2 PCs
pca_2 = PCA(n_components=2).fit(x_train_scaled)
x_train_pca_2 = pca_2.transform(x_train_scaled)
x_test_pca_2 = pca_2.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=500, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 5 PCs
pca_5 = PCA(n_components=5).fit(x_train_scaled)
x_train_pca_2 = pca_5.transform(x_train_scaled)
x_test_pca_2 = pca_5.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=500, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 10 PCs
pca_10 = PCA(n_components=10).fit(x_train_scaled)
x_train_pca_2 = pca_10.transform(x_train_scaled)
x_test_pca_2 = pca_10.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=500, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 15 PCs
pca_15 = PCA(n_components=15).fit(x_train_scaled)
x_train_pca_2 = pca_15.transform(x_train_scaled)
x_test_pca_2 = pca_15.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=500, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 20 PCs
pca_20 = PCA(n_components=20).fit(x_train_scaled)
x_train_pca_2 = pca_20.transform(x_train_scaled)
x_test_pca_2 = pca_20.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=500, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 50 PCs
pca_50 = PCA(n_components=50).fit(x_train_scaled)
x_train_pca_2 = pca_50.transform(x_train_scaled)
x_test_pca_2 = pca_50.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=10000, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

In [ ]:
#Using 100 PCs
pca_100 = PCA(n_components=100).fit(x_train_scaled)
x_train_pca_2 = pca_100.transform(x_train_scaled)
x_test_pca_2 = pca_100.transform(x_test_scaled)

print('Original dimensions:', x_train_scaled.shape)
print('PCA dimensions:     ', x_train_pca_2.shape)

#Training a logistic regression model
logistic_pca_2 = LogisticRegressionCV(max_iter=10000, cv=10, multi_class='multinomial').fit(x_train_pca_2, y2_train)

#Predict
y_pred_train = logistic_pca_2.predict(x_train_pca_2)
y_pred_test = logistic_pca_2.predict(x_test_pca_2)

#Perfromance Evaluation
train_score = accuracy_score(y2_train, y_pred_train)*100
test_score = accuracy_score(y2_test, y_pred_test)*100
print("Training Set Accuracy:",str(train_score)+'%')
print("Testing Set Accuracy:",str(test_score)+'%')

## Calculate classification accuracies

In [ ]:
ALL_t = np.where(y2_test == 0)
ALL_b = np.where(y2_test == 1)
AHL = np.where(y2_test == 2)
y2_test_2 = np.array(y2_test)

In [ ]:
all_t_test_score = accuracy_score(y2_test_2[ALL_t], y_pred_test[ALL_t])*100
all_b_test_score = accuracy_score(y2_test_2[ALL_b], y_pred_test[ALL_b])*100
ahl_test_score = accuracy_score(y2_test_2[AHL], y_pred_test[AHL])*100
print("ALL T Accuracy:",str(all_t_test_score)+'%')
print("ALL B Accuracy:",str(all_b_test_score)+'%')
print("AHL Accuracy:",str(ahl_test_score)+'%')